Importing necessary libraries

In [44]:
import gizmo_analysis as gizmo
import utilities as ut
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import pandas as pd
import os

Obtaining snapshot details

In [45]:
# Specifying simulation directory and the directory to save results in
wdir = str(input('Enter simulation directory path: '))

# Specifying snapshot index
snap_index = int(input('Enter snapshot index: '))

# Creating a directory for spatial analysis

sdir = wdir + 'spatial_analysis/'

if not os.path.exists(sdir):
    os.makedirs(sdir)

# Creating a snapshot for the directory
sdir_snap = wdir + 'spatial_analysis/' + str(snap_index) + '/'

if not os.path.exists(sdir_snap):
    os.makedirs(sdir_snap)

# Importing data from the snapshot
part = gizmo.io.Read.read_snapshots(['star', 'gas', 'dark'], 'index', snap_index, assign_hosts_rotation = True, 
                                    simulation_directory = wdir)

# Getting halo properties
halo_properties = ut.particle.get_halo_properties(part, 'all')

Enter simulation directory path: /Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Data/m10q_res16000_test/
Enter snapshot index: 600

# in utilities.simulation.Snapshot():
* reading:  Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Data/m10q_res16000_test/snapshot_times.txt

  using snapshot index = 600, redshift = 0.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Data/m10q_res16000_test/output/snapshot_600.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 260408 particles
    dark2     (id = 2): 2281573 particles
    gas       (id = 0): 259203 particles
    star      (id = 4): 1205 particles
    blackhole (id = 5): 0 particles

* reading species: ['star', 'gas', 'dark']
* reading particles from:
    Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Data/m10q_res16000_test/output/snapshot_600.hdf5

! cannot find MUSIC config file:  Users/thepoetoftwilight/Documents

In [46]:
r_vir = 0.1*halo_properties['radius']

Specifying list of metals

In [47]:
metals = ['c','n','o','ne','mg','si','s','ca','fe']

Finding stellar profile properties

In [48]:
distances = part['star'].prop('host.distance.total')

num_stars = len(distances)
print(num_stars)

print(distances)

form_times = part['star'].prop('form.time')
print(form_times)

ages = part['star'].prop('age')
print(ages)

form_masses = part['star'].prop('form.mass')
print(form_masses)

curr_masses = part['star'].prop('mass')
print(curr_masses)

total_abundances = part['star'].prop('metallicity.total')
print(total_abundances)

metal_abundances = {}

for m in metals:
    metal_abundances[m] = part['star'].prop('metallicity.{}'.format(m))
    print(metal_abundances[m])

1205
[ 1.36661734  1.49642784  1.11235743 ...  1.47368261  1.39549293
 10.79931902]
[1.97235653 3.3535738  7.0060697  ... 8.66450675 4.11862962 1.76017632]
[11.82639036 10.44517309  6.79267719 ...  5.13424013  9.68011727
 12.03857056]

# in gizmo_analysis.gizmo_star.MassLoss():
* generating 2-D spline to compute stellar mass loss from age + metallicity
  number of age bins = 22
  number of metallicity bins = 26
[17640.92333396 19051.5768098  15919.18292288 ... 16401.85571468
 17330.88211748 15620.19793123]
[12005.636 13038.455 11113.028 ... 11597.009 11901.963 10621.891]
[-2.460777  -2.1569438 -1.5293121 ... -1.4703783 -1.8204105 -1.834552 ]
[-2.2822006 -2.3026814 -1.3943698 ... -1.4124699 -1.3504798 -2.0164683]
[-2.452191  -2.6329303 -1.6023697 ... -1.6473573 -1.4944829 -2.3911932]
[-2.6622427 -2.0491965 -1.5259714 ... -1.427635  -2.1352344 -1.7105858]
[-2.6437945 -1.9822705 -1.4626676 ... -1.3625807 -2.0746264 -1.6424248]
[-2.8441253 -2.2227387 -1.7015226 ... -1.6029406 -2.3077617 -1

Writing stellar profiles to a .csv file

In [52]:
stellar_dict = {'distance': [], 'form_time': [], 'age': [], 
                'form_mass': [], 'curr_mass': [], 
                'total_abundance': []}

for m in metals:
    stellar_dict['[{}/H]'.format(m.title())] = []

for i in range(0, num_stars):
    
    if(distances[i] < 0.1*r_vir):
        
        stellar_dict['distance'].append(distances[i])
        stellar_dict['form_time'].append(form_times[i])
        stellar_dict['age'].append(ages[i])
        stellar_dict['form_mass'].append(form_masses[i])
        stellar_dict['curr_mass'].append(curr_masses[i])
        stellar_dict['total_abundance'].append(total_abundances[i])
        
        for m in metals:
            stellar_dict['[{}/H]'.format(m.title())].append(metal_abundances[m][i])


stellar_df = pd.DataFrame(stellar_dict)
stellar_df.to_csv(sdir_snap + 'ISM_stellar_profile.csv')